--- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

    30373
    25512
    65332
    33549
    35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

- The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
- The top-middle 5 is visible from the top and right.
- The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
- The left-middle 5 is visible, but only from the right.
- The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
- The right-middle 3 is visible from the right.
- In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [76]:
import pandas as pd

In [77]:
with open("input.txt", "r") as f:
    lines = f.read().splitlines()

trees = pd.DataFrame([list(digit) for digit in lines])

In [78]:
rows_count, columns_counts = trees.shape
trees_count = rows_count * columns_counts
trees_count

9801

In [79]:
trees

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,2,2,2,3,2,2,2,1,3,0,...,1,1,1,2,3,0,0,0,4,0
1,1,1,0,2,0,0,1,3,4,4,...,2,0,4,3,2,3,4,0,0,1
2,1,3,0,2,4,1,1,2,2,3,...,5,2,4,5,1,4,0,0,3,3
3,0,4,4,2,0,1,1,5,5,3,...,0,4,2,3,0,3,3,0,4,3
4,3,4,1,2,4,2,0,4,3,0,...,2,1,4,2,4,0,5,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,4,1,2,1,1,5,3,3,4,4,...,1,1,5,4,3,2,2,2,0,4
95,0,4,1,3,3,5,4,5,5,1,...,1,0,3,4,0,5,0,2,4,1
96,0,3,0,0,2,3,1,0,2,5,...,1,1,3,4,5,4,1,0,0,2
97,4,0,4,4,4,3,5,3,2,4,...,0,5,0,5,5,4,0,4,3,0


In [80]:
def is_invisible(tree_height, row, column):
    """We are counting invisible trees to then substract them from the total number of trees in the forest.
    Look up, right, left and down for trees of same or greater height. If this happens in all directions, then the tree is invisible.
    """
    invisible_direction_count = 0

    # Look up
    for i in range(row-1, -1, -1):
        if trees.iloc[i, column] >= tree_height:
            # print(f"Up: {trees.iloc[i, column]} makes {tree_height} invisible.")
            invisible_direction_count += 1
            break

    # Look down
    for i in range(row+1, rows_count):
        if trees.iloc[i, column] >= tree_height:
            # print(f"Down: {trees.iloc[i, column]} makes {tree_height} invisible.")
            invisible_direction_count += 1
            break
    
    # Look left
    for i in range(column-1, -1, -1):
        if trees.iloc[row, i] >= tree_height:
            # print(f"Left: {trees.iloc[row, i]} makes {tree_height} invisible.")
            invisible_direction_count += 1
            break
    
    # Look right
    for i in range(column+1, columns_counts):
        if trees.iloc[row, i] >= tree_height:
            # print(f"Right: {trees.iloc[row, i]} makes {tree_height} invisible.")
            invisible_direction_count += 1
            break
    
    if invisible_direction_count == 4:
        return True
    else:
        return False    

In [81]:
invisible_count = 0

for i in range(1, rows_count-1):
    for j in range(1, columns_counts-1):
        if is_invisible(trees.iloc[i, j], i, j):
            invisible_count += 1

Question: how many trees are VISIBLE from outside the grid?

In [82]:
answer = trees_count - invisible_count
print(f"{answer} trees are visible.")

1533 trees are visible.


--- Part Two ---

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

    30373
    25512
    65332
    33549
    35390

- Looking up, its view is not blocked; it can see 1 tree (of height 3).
- Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
- Looking right, its view is not blocked; it can see 2 trees.
- Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

    30373
    25512
    65332
    33549
    35390

- Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
- Looking left, its view is not blocked; it can see 2 trees.
- Looking down, its view is also not blocked; it can see 1 tree.
- Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [87]:
def get_scenic_score(tree_height, row, column):
    """We are counting invisible trees to then substract them from the total number of trees in the forest.
    Look up, right, left and down for trees of same or greater height. If this happens in all directions, then the tree is invisible.
    """
    up = 0
    down = 0
    left = 0
    right = 0

    # Look up
    for i in range(row-1, -1, -1):
        if trees.iloc[i, column] < tree_height:
            # print(f"Up: {trees.iloc[i, column]} is visible from tree {tree_height}.")
            up += 1
        else:
            up += 1
            break
    
    # Look down
    for i in range(row+1, rows_count):
        if trees.iloc[i, column] < tree_height:
            # print(f"Down: {trees.iloc[i, column]} is visible from tree {tree_height}.")
            down += 1
        else:
            down += 1
            break
    
    # Look left
    for i in range(column-1, -1, -1):
        if trees.iloc[row, i] < tree_height:
            # print(f"Left: {trees.iloc[row, i]} is visible from tree {tree_height}.")
            left += 1
        else:
            left += 1
            break

    # Look right
    for i in range(column+1, columns_counts):
        if trees.iloc[row, i] < tree_height:
            # print(f"Right: {trees.iloc[row, i]} is visible from tree {tree_height}.")
            right += 1
        else:
            right += 1
            break

    scenic_score = up * down * left * right
    return scenic_score

In [88]:
scenic_scores = []

for i in range(1, rows_count-1):
    for j in range(1, columns_counts-1):
        # print(f"Tree {trees.iloc[i, j]} at ({i}, {j}) has a scenic score of: ", end="")
        # print(get_scenic_score(trees.iloc[i, j], i, j))
        scenic_scores.append(get_scenic_score(trees.iloc[i, j], i, j))

print(f"{max(scenic_scores)} is the highest scenic score.")

345744 is the highest scenic score.
